In [1]:
!nvidia-smi

Wed Apr 19 09:26:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:73:00.0  On |                  Off |
| 30%   41C    P8    29W / 230W |  22663MiB / 24564MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install kaggle
# !mkdir /root/.kaggle

# !cp /content/drive/MyDrive/kaggle/kaggle.json /root/.kaggle/kaggle.json
# !chmod 600 /root/.kaggle/kaggle.json

In [4]:
# !kaggle datasets download -d chaudharypriyanshu/bci-42a-dataset
# ! unzip /content/bci-42a-dataset.zip

In [5]:
# !pip install neural_structured_learning

In [28]:
class CFG:
  results_path="/workspace/SAMAD_PRAGATI_PRATYUSH_KARINA/EEG-ATCNet/results/"
#   data_path='/kaggle/input/bci-42a-dataset/'
  data_path="/workspace/SAMAD_PRAGATI_PRATYUSH_KARINA/EEG-ATCNet/BCICIV_2a_gdf/"
  n_classes=4
  n_sub=9
  n_channels=22
  isStandard=True
  LOSO=False
  verbose=0
  batch_size=64
  epochs=1000
  patience=300
  lr=0.0007
  LearnCurves=True
  n_train=1
  model_name='ATCNet'


In [29]:
from tqdm import tqdm
import pandas as pd
from sklearn.cluster import KMeans
import random
random.seed(1001)
def channels(X_train):
  train=pd.DataFrame(columns=range(0,22))
  for i in tqdm(range(0,X_train.shape[0])):
    t=X_train[i,0]
    corr=np.corrcoef(t)
    train=pd.concat([train,pd.DataFrame(corr,columns=range(0,22))],axis=0)
  kmeans = KMeans(n_clusters=3,tol=0,random_state=0,algorithm='elkan').fit(train.T.values)
  channellabels=kmeans.labels_ 
  return [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2] 

def augment(X_train,y_train,counts=0):
  data=[]
  labels=[]
  for i in [1,2,3,4]:
    result = np.where(y_train == i)[0]
    temp=np.copy(X_train[result,:,:,:])
    print('result:',temp.shape)
    channellabels=channels(temp)
    done=set()
    for turn in range(counts):
      clst=random.randint(0,2)
      cluster=np.where(channellabels==clst)[0]
      r1=random.randint(0, 71)
      r2=random.randint(0, 71)
      if((r1==r2) or ((r1,r2,clst) in done)):
        continue
      done.add((r1,r2,clst))

      transform1=np.copy(temp[r1,:,:,:])
      transform2=np.copy(temp[r2,:,:,:])

      transform1[:,cluster,:]=transform2[:,cluster,:]
      data.append(transform1)
      labels.append(i)

  return data,labels

import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler,MinMaxScaler

#%%
def load_data_LOSO (data_path, subject): 

    X_train, y_train = [], []
    for sub in range (0,9):
        path = data_path
        
        X1, y1 = load_data(sub+1, True ,path)
        X2, y2 = load_data(sub+1, False ,path)
        X = np.concatenate((X1, X2), axis=0)
        y = np.concatenate((y1, y2), axis=0)
                   
        if (sub == subject):
            X_test = X
            y_test = y
        elif (X_train == []):
            X_train = X
            y_train = y
        else:
            X_train = np.concatenate((X_train, X), axis=0)
            y_train = np.concatenate((y_train, y), axis=0)

    return X_train, y_train, X_test, y_test

#%%
def load_data(data_path, subject, training, all_trials = True):

	n_channels = 22
	n_tests = 6*48 	
	window_Length = 7*250 

	class_return = np.zeros(n_tests)
	data_return = np.zeros((n_tests, n_channels, window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(data_path+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(data_path+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2= [a_data1[0,0]]
		a_data3= a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_artifacts = a_data3[5]

		for trial in range(0,a_trial.size):
 			if(a_artifacts[trial] != 0 and not all_trials):
 			    continue

 			data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+window_Length),:22])
 			class_return[NO_valid_trial] = int(a_y[trial])
 			NO_valid_trial +=1

	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

#%%
def standardize_data(X_train, X_test, channels): 

    for j in range(channels):
          scaler = StandardScaler()
          scaler.fit(X_train[:,0, j, :])
          X_train[:,0, j, :] = scaler.transform(X_train[:,0, j, :])
          X_test[:,0, j, :] = scaler.transform(X_test[:,0, j, :])

    return X_train, X_test

#%%
def get_data(path, subject, LOSO = False, isStandard = True):

    fs = 250  
    t1 = int(1.5*fs) 
    t2 = int(6*fs)   
    T = t2-t1        
    
    if LOSO:

        X_train, y_train, X_test, y_test = load_data_LOSO(path, subject)
    else:
  
        path = path
        X_train, y_train = load_data(path, subject+1, True)
        X_test, y_test = load_data(path, subject+1, False)

    N_tr, N_ch, _ = X_train.shape 
    X_train = X_train[:, :, t1:t2].reshape(N_tr,1, N_ch, T)

    X_train=CAR(X_train)
    
    data,label=augment(X_train,y_train,40)
    X_train=np.concatenate((X_train,data),axis=0)
    y_train=np.concatenate((y_train,label),axis=0)

    y_train_onehot = (y_train-1).astype(int)
    y_train_onehot = to_categorical(y_train_onehot)
 
    N_test, N_ch, _ = X_test.shape 
    X_test = X_test[:, :, t1:t2].reshape(N_test,1, N_ch, T)
    y_test_onehot = (y_test-1).astype(int)
    y_test_onehot = to_categorical(y_test_onehot)	
    X_test=CAR(X_test)

    print('TRAINING_SHAPE:',X_train.shape)


    if (isStandard == True):
        X_train, X_test = standardize_data(X_train, X_test, N_ch)


    return X_train, y_train, y_train_onehot, X_test, y_test, y_test_onehot

def CAR(trials):
    sigma = np.mean(trials, axis=2, keepdims=True)
    tnew = (trials - sigma)
    return tnew


In [30]:
#@title Default title text
import math
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense
from tensorflow.keras.layers import multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from tensorflow.keras.layers import Dropout, MultiHeadAttention, LayerNormalization
from tensorflow.keras import backend as K
import math
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense
from tensorflow.keras.layers import multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from tensorflow.keras.layers import Dropout, MultiHeadAttention, LayerNormalization,SeparableConv1D,AveragePooling1D,MaxPool1D
from tensorflow.keras import backend as K
from tensorflow.nn import avg_pool1d

#%% Creat and return the attention model
def get_attention_model(net, attention_model): 
    expanded_axis = 3 # defualt = 3
    if attention_model == 'mha':   # Multi-head self attention layer 
        net = mha_block(net)
    elif attention_model == 'mhla':  # Multi-head local self-attention layer 
        net = mha_block(net, vanilla = False)
    elif attention_model == 'se':   # Squeeze-and-excitation layer
        if(len(net.shape) < 4):
            net = tf.expand_dims(net, axis=expanded_axis)
        net = se_block(net, ratio=8)
    elif attention_model == 'cbam': # Convolutional block attention module
        if(len(net.shape) < 4):
            net = tf.expand_dims(net, axis=expanded_axis)
        net = cbam_block(net, ratio=8)
    else:
        raise Exception("'{}' is not supported attention module!".format(attention_model))
        
    if(len(net.shape) == 4):
        net = K.squeeze(net, expanded_axis)
    return net


#%% Multi-head self Attention (MHA) block
def mha_block(input_feature, key_dim=8, num_heads=2, dropout = 0.5, vanilla = True):
    """Multi Head self Attention (MHA) block.     
       
    Here we include two types of MHA blocks: 
            The original multi-head self-attention as described in https://arxiv.org/abs/1706.03762
            The multi-head local self attention as described in https://arxiv.org/abs/2112.13492v1
    """    
    # Layer normalization
    x = LayerNormalization(epsilon=1e-6)(input_feature)
    
    if vanilla:
        # Create a multi-head attention layer as described in 
        # 'Attention Is All You Need' https://arxiv.org/abs/1706.03762
        x = MultiHeadAttention(key_dim = key_dim, num_heads = num_heads, dropout = dropout)(x, x)
    else:
        # Create a multi-head local self-attention layer as described in 
        # 'Vision Transformer for Small-Size Datasets' https://arxiv.org/abs/2112.13492v1
        
        # Build the diagonal attention mask
        NUM_PATCHES = input_feature.shape[1]
        diag_attn_mask = 1 - tf.eye(NUM_PATCHES)
        diag_attn_mask = tf.cast([diag_attn_mask], dtype=tf.int8)
        
        # Create a multi-head local self attention layer.
        x = MultiHeadAttention_LSA(key_dim = key_dim, num_heads = num_heads, dropout = dropout)(
            x, x, attention_mask = diag_attn_mask)
        
    x = Dropout(0.3)(x)
    # Skip connection
    mha_feature = Add()([input_feature, x])
    
    return mha_feature


#%% Multi head self Attention (MHA) block: Locality Self Attention (LSA)
class MultiHeadAttention_LSA(tf.keras.layers.MultiHeadAttention):
    """local multi-head self attention block
     
     Locality Self Attention as described in https://arxiv.org/abs/2112.13492v1
     This implementation is taken from  https://keras.io/examples/vision/vit_small_ds/ 
    """    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # The trainable temperature term. The initial value is the square 
        # root of the key dimension.
        self.tau = tf.Variable(math.sqrt(float(self._key_dim)), trainable=True)

    def _compute_attention(self, query, key, value, attention_mask=None, training=None):
        query = tf.multiply(query, 1.0 / self.tau)
        attention_scores = tf.einsum(self._dot_product_equation, key, query)
        attention_scores = self._masked_softmax(attention_scores, attention_mask)
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )
        attention_output = tf.einsum(
            self._combine_equation, attention_scores_dropout, value
        )
        return attention_output, attention_scores


#%% Squeeze-and-excitation block
def se_block(input_feature, ratio=8):
	"""Squeeze-and-Excitation(SE) block.
    
	As described in https://arxiv.org/abs/1709.01507
    The implementation is taken from https://github.com/kobiso/CBAM-keras
	"""
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]

	se_feature = GlobalAveragePooling2D()(input_feature)
	se_feature = Reshape((1, 1, channel))(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)
	se_feature = Dense(channel // ratio,
					   activation='relu',
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	assert se_feature.shape[1:] == (1,1,channel//ratio)
	se_feature = Dense(channel,
					   activation='sigmoid',
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)
	if K.image_data_format() == 'channels_first':
		se_feature = Permute((3, 1, 2))(se_feature)

	se_feature = multiply([input_feature, se_feature])
	return se_feature


#%% Convolutional block attention module
def cbam_block(cbam_feature, ratio=8):
	""" Convolutional Block Attention Module(CBAM) block.
    
	As described in https://arxiv.org/abs/1807.06521
    The implementation is taken from https://github.com/kobiso/CBAM-keras
	"""
	
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature

def channel_attention(input_feature, ratio=8):
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
# 	channel = input_feature._keras_shape[channel_axis]
	channel = input_feature.shape[channel_axis]
    
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
	kernel_size = 7
	
	if K.image_data_format() == "channels_first":
		channel = input_feature.shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature.shape[-1]
		cbam_feature = input_feature
	
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool.shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool.shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat.shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					strides=1,
					padding='same',
					activation='sigmoid',
					kernel_initializer='he_normal',
					use_bias=False)(concat)	
	assert cbam_feature.shape[-1] == 1
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
		
	return multiply([input_feature, cbam_feature])
		
	

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, AveragePooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization, LayerNormalization, Flatten 
from tensorflow.keras.layers import Add, Concatenate, Lambda, Input, Permute
from tensorflow.keras.constraints import max_norm


def ATCNet(n_classes, in_chans = 22, in_samples = 1000, n_windows = 3, attention = None, 
           eegn_F1 = 16, eegn_D = 2, eegn_kernelSize = 64, eegn_poolSize = 8, eegn_dropout=0.3, 
           tcn_depth = 2, tcn_kernelSize = 4, tcn_filters = 32, tcn_dropout = 0.3, 
           tcn_activation = 'elu', fuse = 'average'):

    input_1 = Input(shape = (1,in_chans, in_samples))
    input_2 = Permute((3,2,1))(input_1) 
    regRate=.25
    numFilters = eegn_F1
    F2 = numFilters*eegn_D

    block1 = Conv_block(input_layer = input_2, F1 = eegn_F1, D = eegn_D, poolSize = eegn_poolSize, Filters=[16,32,64],
                        in_chans = in_chans, dropout = eegn_dropout)
    block1 = Lambda(lambda x: x[:,:,-1,:])(block1)
    print('ATCNet block1 shape:',block1.shape)

    sw_concat = []  
    embed = []
    for i in range(n_windows):
        st = i
        end = block1.shape[1]-n_windows+i+1
        block2 = block1[:, st:end, :]
        
        if attention is not None:
            block2 = get_attention_model(block2, attention)

        block3 = TCN_block(input_layer = block2, input_dimension = F2, depth = tcn_depth,
                            kernel_size = tcn_kernelSize, filters = tcn_filters, 
                            dropout = tcn_dropout, activation = tcn_activation)

        embed.append(block3)
        
        block3 = Lambda(lambda x: x[:,-1,:])(block3)

        if(fuse == 'average'):
            sw_concat.append(Dense(n_classes, kernel_constraint = max_norm(regRate))(block3))
        elif(fuse == 'concat'):
            if i == 0:
                sw_concat = block3
            else:
                sw_concat = Concatenate()([sw_concat, block3])
                
    if(fuse == 'average'):
        if len(sw_concat) > 1:
            sw_concat = tf.keras.layers.Average()(sw_concat[:])
        else:
            sw_concat = sw_concat[0]
    elif(fuse == 'concat'):
        sw_concat = Dense(n_classes, kernel_constraint = max_norm(regRate))(sw_concat)
            
    embed = Concatenate(name = 'concit')(embed)
    softmax = Activation('softmax', name = 'softmax')(sw_concat)
    
    return Model(inputs = input_1, outputs = [softmax,embed])


def Conv_block(input_layer, F1=4, Filters=[1,2,4,8,16,32,64], poolSize=7, D=2, in_chans=22, dropout=0.1):

    F2= F1*D
    out_sep=[]
    for i,kernel_size in enumerate(Filters):
      block1 = Conv2D(8, (kernel_size,1), padding = 'same',data_format='channels_last',use_bias = False)(input_layer)
      out_sep.append(block1)
      print('kernel_size:',block1.shape)

    block1=Concatenate(axis=-1)(out_sep)
    block1 = BatchNormalization(axis = -1)(block1)


    print('block1:',block1.shape)
    block2 = DepthwiseConv2D((1, in_chans), use_bias = False, 
                                    depth_multiplier = D,
                                    data_format='channels_last',
                                    depthwise_constraint = max_norm(1.))(block1)
    block2 = BatchNormalization(axis = -1)(block2)
    block2 = Activation('elu')(block2)
    print('block2:',block2.shape)
    block2 = AveragePooling2D((7,1),data_format='channels_last')(block2)
    block2 = Dropout(dropout)(block2)
    print('After pooling block2:',block2.shape)
    block3 = Conv2D(F2,(16,1),
                            data_format='channels_last',
                            use_bias = False, padding = 'same')(block2)
    block3 = BatchNormalization(axis = -1)(block3)
    block3 = Activation('elu')(block3)
    print('block3:',block3.shape)
    block3 = AveragePooling2D((poolSize,1),data_format='channels_last')(block3)
    block3 = Dropout(dropout)(block3)
    print('block3:',block3.shape)
    return block3

def TCN_block(input_layer,input_dimension,depth,kernel_size,filters,dropout,activation='relu'):
    
    block = SeparableConv1D(filters,kernel_size=kernel_size,dilation_rate=1,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(input_layer)
    block = BatchNormalization()(block)
    block = Activation(activation)(block)
    block = Dropout(dropout)(block)
    block = SeparableConv1D(filters,kernel_size=kernel_size,dilation_rate=1,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(block)
    block = BatchNormalization()(block)
    block = Activation(activation)(block)
    block = Dropout(dropout)(block)
    if(input_dimension != filters):
        conv = SeparableConv1D(filters,kernel_size=1,padding='same')(input_layer)
        added = Add()([block,conv])
    else:
        added = Add()([block,input_layer])
    out = Activation(activation)(added)
    
    for i in range(depth-1):
        block = SeparableConv1D(filters,kernel_size=kernel_size,dilation_rate=2**(i+1),activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(out)
        block = BatchNormalization()(block)
        block = Activation(activation)(block)
        block = Dropout(dropout)(block)
        block = SeparableConv1D(filters,kernel_size=kernel_size,dilation_rate=2**(i+1),activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(block)
        block = BatchNormalization()(block)
        block = Activation(activation)(block)
        block = Dropout(dropout)(block)      
        added = Add()([block, out])
        out = Activation(activation)(added)
      

    return out

def  Xception_block(input_layer,filters=8,layers=[8,16,32],dilation_rate=1,dropout=0.1,activation='relu'):
    
    print('Xception input:',input_layer.shape)
    block1= Conv1D(filters,kernel_size=4,dilation_rate=dilation_rate,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(input_layer)
    output=[]
    for _,kernel in enumerate(layers):
          out_sep=tf.keras.layers.SeparableConv1D(filters,kernel,padding='causal',dilation_rate=dilation_rate)(block1)
          out_sep = BatchNormalization()(out_sep)
          output.append(out_sep)
    output=Concatenate(axis=-1)(output)
    output = Activation(activation)(output) 

    print('Xception parallel separable conv shape:',output.shape)
    block2=Conv1D(filters,kernel_size=5,dilation_rate=dilation_rate,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(input_layer)  
    block2 = BatchNormalization()(block2)
    block2 = Activation(activation)(block2)                             
    print('Xception block2 shape:',block2.shape)
    output=Concatenate(axis=-1)([output,block2])
    print('Xception output shape:',output.shape)  
    return output

def TCNet_Fusion(n_classes,Chans=22, Samples=1000, layers=2, kernel_s=4, filt=12,
                 dropout=0.3, activation='elu', F1=24, D=2, kernLength=32, dropout_eeg=0.3):

    input1 = Input(shape = (1,Chans, Samples))
    input2 = Permute((3,2,1))(input1)
    regRate=.25

    numFilters = F1
    F2= numFilters*D
    
    EEGNet_sep = EEGNet(input_layer=input2,F1=F1,kernLength=kernLength,D=D,Chans=Chans,dropout=dropout_eeg)
    block2 = Lambda(lambda x: x[:,:,-1,:])(EEGNet_sep)
    FC = Flatten()(block2) 

    outs = TCN_block(input_layer=block2,input_dimension=F2,depth=layers,kernel_size=kernel_s,filters=filt,dropout=dropout,activation=activation)

    Con1 = Concatenate()([block2,outs]) 
    out = Flatten()(Con1) 
    Con2 = Concatenate()([out,FC]) 
    dense        = Dense(n_classes, name = 'dense',kernel_constraint = max_norm(regRate))(Con2)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1,outputs=softmax)


def EEGTCNet(n_classes, Chans=22, Samples=1000, layers=2, kernel_s=4, filt=12, dropout=0.3, activation='elu', F1=8, D=2, kernLength=32, dropout_eeg=0.2):

    input1 = Input(shape = (1,Chans, Samples))
    input2 = Permute((3,2,1))(input1)
    regRate=.25
    numFilters = F1
    F2= numFilters*D

    EEGNet_sep = EEGNet(input_layer=input2,F1=F1,kernLength=kernLength,D=D,Chans=Chans,dropout=dropout_eeg)
    block2 = Lambda(lambda x: x[:,:,-1,:])(EEGNet_sep)
    outs = TCN_block(input_layer=block2,input_dimension=F2,depth=layers,kernel_size=kernel_s,filters=filt,dropout=dropout,activation=activation)
    out = Lambda(lambda x: x[:,-1,:])(outs)
    dense        = Dense(n_classes, name = 'dense',kernel_constraint = max_norm(regRate))(out)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1,outputs=softmax)

def EEGNeX_8_32(n_timesteps, n_features, n_outputs):

    model = Sequential()
    model.add(Input(shape=(1, n_features, n_timesteps)))

    model.add(Conv2D(filters=8, kernel_size=(1, 32), use_bias = False, padding='same', data_format="channels_first"))
    model.add(LayerNormalization())
    model.add(Activation(activation='elu'))
    model.add(Conv2D(filters=32, kernel_size=(1, 32), use_bias = False, padding='same', data_format="channels_first"))
    model.add(LayerNormalization())
    model.add(Activation(activation='elu'))

    model.add(DepthwiseConv2D(kernel_size=(n_features, 1), depth_multiplier=2, use_bias = False, depthwise_constraint=max_norm(1.), data_format="channels_first"))
    model.add(LayerNormalization())
    model.add(Activation(activation='elu'))
    model.add(AveragePooling2D(pool_size=(1, 4), padding='same', data_format="channels_first"))
    model.add(Dropout(0.5))

    
    model.add(Conv2D(filters=32, kernel_size=(1, 16), use_bias = False, padding='same', dilation_rate=(1, 2), data_format='channels_first'))
    model.add(LayerNormalization())
    model.add(Activation(activation='elu'))
    
    model.add(Conv2D(filters=8, kernel_size=(1, 16), use_bias = False, padding='same', dilation_rate=(1, 4),  data_format='channels_first'))
    model.add(LayerNormalization())
    model.add(Activation(activation='elu'))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(n_outputs, kernel_constraint=max_norm(0.25)))
    model.add(Activation(activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 

def EEGNet_classifier(n_classes, Chans=22, Samples=1000, F1=8, D=2, kernLength=64, dropout_eeg=0.25):
    input1 = Input(shape = (1,Chans, Samples))   
    input2 = Permute((3,2,1))(input1) 
    regRate=.25

    eegnet = EEGNet(input_layer=input2, F1=F1, kernLength=kernLength, D=D, Chans=Chans, dropout=dropout_eeg)
    eegnet = Flatten()(eegnet)
    dense = Dense(n_classes, name = 'dense',kernel_constraint = max_norm(regRate))(eegnet)
    softmax = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

def EEGNet(input_layer, F1=8, kernLength=64, D=2, Chans=22, dropout=0.25):
    F2= F1*D
    block1 = Conv2D(F1, (kernLength, 1), padding = 'same',data_format='channels_last',use_bias = False)(input_layer)
    block1 = BatchNormalization(axis = -1)(block1)
    block2 = DepthwiseConv2D((1, Chans), use_bias = False, 
                                    depth_multiplier = D,
                                    data_format='channels_last',
                                    depthwise_constraint = max_norm(1.))(block1)
    block2 = BatchNormalization(axis = -1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((8,1),data_format='channels_last')(block2)
    block2 = Dropout(dropout)(block2)
    block3 = SeparableConv2D(F2, (16, 1),
                            data_format='channels_last',
                            use_bias = False, padding = 'same')(block2)
    block3 = BatchNormalization(axis = -1)(block3)
    block3 = Activation('elu')(block3)
    block3 = AveragePooling2D((8,1),data_format='channels_last')(block3)
    block3 = Dropout(dropout)(block3)
    return block3

In [34]:
#@title Default title text
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score

#%%
def draw_learning_curves(history):
    plt.plot(history.history['softmax_accuracy'])
    plt.plot(history.history['val_softmax_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()
    plt.close()

def draw_confusion_matrix(cf_matrix, sub, results_path):
    # Generate confusion matrix plot
    display_labels = ['Left hand', 'Right hand','Foot','Tongue']
    disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix, 
                                display_labels=display_labels)
    disp.plot()
    disp.ax_.set_xticklabels(display_labels, rotation=12)
    plt.title('Confusion Matrix of Subject: ' + sub )
    plt.savefig(results_path + '/subject_' + sub + '.png')
    plt.show()

def draw_performance_barChart(num_sub, metric, label):
    fig, ax = plt.subplots()
    x = list(range(1, num_sub+1))
    ax.bar(x, metric, 0.5, label=label)
    ax.set_ylabel(label)
    ax.set_xlabel("Subject")
    ax.set_xticks(x)
    ax.set_title('Model '+ label + ' per subject')
    ax.set_ylim([0,1])
    

#%% Training 
def train():
    in_exp=time.time()

    best_models = open(CFG.results_path + "/best models.txt", "w")

    log_write = open(CFG.results_path + "/log.txt", "w")

    perf_allRuns = open(CFG.results_path + "/perf_allRuns.npz", 'wb')
    
    # Initialize variables
    acc = np.zeros((CFG.n_sub, CFG.n_train))
    kappa = np.zeros((CFG.n_sub, CFG.n_train))
    
    # Iteration over subjects
    for sub in [0,3,8]: 
  
        print('\nTraining on subject ', sub+1)
        log_write.write( '\nTraining on subject '+ str(sub+1) +'\n')

        BestSubjAcc = 0 
        bestTrainingHistory = [] 

        X_train, _, y_train_onehot, X_test, _, y_test_onehot = get_data(
            CFG.data_path, sub, CFG.LOSO, CFG.isStandard)

        in_sub = time.time()
        for train in range(CFG.n_train): 

            in_run = time.time()

            filepath = CFG.results_path + '/saved models/run-{}'.format(train+1)
            if not os.path.exists(filepath):
                os.makedirs(filepath)        
            filepath = filepath + '/subject-{}.h5'.format(sub+1)
            
            # Create the model
            model = getModel(CFG.model_name)
            opt=Adam(lr=CFG.lr)

            model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'],loss_weights=[1., 0.0])          
            callbacks = [
                ModelCheckpoint(filepath, monitor='val_softmax_accuracy', verbose=2, 
                                save_best_only=True, save_weights_only=True, mode='max'),
                EarlyStopping(monitor='val_softmax_accuracy', verbose=0, mode='max', patience=CFG.patience+1000),
            ]

            val_data=(X_test,[y_test_onehot, np.zeros((len(y_test_onehot),18,160))])
            history = model.fit(X_train,[y_train_onehot,np.zeros((len(y_train_onehot),18,160))], validation_data=val_data, 
                                epochs=CFG.epochs, batch_size=CFG.batch_size, callbacks=callbacks, verbose=0)

            model.load_weights(filepath)
            test={'input': X_test,'label':y_test_onehot}


            embed_train = model.predict(X_train)[1]
            embed_test = model.predict(X_test)[1]
            
            np.save(f'y_train_sub{sub}.npy',y_train_onehot)
            np.save(f'embed_train_sub{sub}.npy',embed_train)
            np.save(f'embed_test_sub{sub}.npy',embed_test)

            y_pred = model.predict(X_test)[0].argmax(axis=-1)
            labels = y_test_onehot.argmax(axis=-1)
            acc[sub, train]  = accuracy_score(labels, y_pred)
            kappa[sub, train] = cohen_kappa_score(labels, y_pred)

            out_run = time.time()

            info = 'Subject: {}   Train no. {}   Time: {:.1f} m   '.format(sub+1, train+1, ((out_run-in_run)/60))
            info = info + 'Test_acc: {:.4f}   Test_kappa: {:.4f}'.format(acc[sub, train], kappa[sub, train])
            print(info)
            log_write.write(info +'\n')

            if(BestSubjAcc < acc[sub, train]):
                 BestSubjAcc = acc[sub, train]
                 bestTrainingHistory = history

        best_run = np.argmax(acc[sub,:])
        filepath = '/saved models/run-{}/subject-{}.h5'.format(best_run+1, sub+1)+'\n'
        best_models.write(filepath)
        out_sub = time.time()

        info = '----------\n'
        info = info + 'Subject: {}   best_run: {}   Time: {:.1f} m   '.format(sub+1, best_run+1, ((out_sub-in_sub)/60))
        info = info + 'acc: {:.4f}   avg_acc: {:.4f} +- {:.4f}   '.format(acc[sub, best_run], np.average(acc[sub, :]), acc[sub,:].std() )
        info = info + 'kappa: {:.4f}   avg_kappa: {:.4f} +- {:.4f}'.format(kappa[sub, best_run], np.average(kappa[sub, :]), kappa[sub,:].std())
        info = info + '\n----------'
        print(info)
        log_write.write(info+'\n')
        if (CFG.LearnCurves):
            print('Plot Learning Curves ....... ')
            draw_learning_curves(bestTrainingHistory)

    out_exp = time.time()
    info = '\nTime: {:.1f} h   '.format( (out_exp-in_exp)/(60*60) )
    print(info)

    log_write.write(info+'\n')

    np.savez(perf_allRuns, acc = acc, kappa = kappa)

    best_models.close()   
    log_write.close() 
    perf_allRuns.close() 

def test(model, dataset_conf, results_path, allRuns = True):

    log_write = open(results_path + "/log.txt", "a")

    best_models = open(results_path + "/best models.txt", "r")   

    n_classes = dataset_conf.get('n_classes')
    n_sub = dataset_conf.get('n_sub')
    data_path = dataset_conf.get('data_path')
    isStandard = dataset_conf.get('isStandard')
    LOSO = dataset_conf.get('LOSO')

    acc_bestRun = np.zeros(n_sub)
    kappa_bestRun = np.zeros(n_sub)  
    cf_matrix = np.zeros([n_sub, n_classes, n_classes])

    if(allRuns): 

        perf_allRuns = open(results_path + "/perf_allRuns.npz", 'rb')
        perf_arrays = np.load(perf_allRuns)
        acc_allRuns = perf_arrays['acc']
        kappa_allRuns = perf_arrays['kappa']
    

    for sub in range(n_sub):

        _, _, _, X_test, _, y_test_onehot = get_data(data_path, sub, LOSO, isStandard)

        filepath = best_models.readline()
        model.load_weights(results_path + filepath[:-1])

        y_pred = model.predict(X_test).argmax(axis=-1)

        labels = y_test_onehot.argmax(axis=-1)
        acc_bestRun[sub] = accuracy_score(labels, y_pred)
        kappa_bestRun[sub] = cohen_kappa_score(labels, y_pred)

        cf_matrix[sub, :, :] = confusion_matrix(labels, y_pred, normalize='pred')
        draw_confusion_matrix(cf_matrix[sub, :, :], str(sub+1), results_path)

        info = 'Subject: {}   best_run: {:2}  '.format(sub+1, (filepath[filepath.find('run-')+4:filepath.find('/sub')]) )
        info = info + 'acc: {:.4f}   kappa: {:.4f}   '.format(acc_bestRun[sub], kappa_bestRun[sub] )
        if(allRuns): 
            info = info + 'avg_acc: {:.4f} +- {:.4f}   avg_kappa: {:.4f} +- {:.4f}'.format(
                np.average(acc_allRuns[sub, :]), acc_allRuns[sub,:].std(),
                np.average(kappa_allRuns[sub, :]), kappa_allRuns[sub,:].std() )
        print(info)
        log_write.write('\n'+info)
      
    info = '\nAverage of {} subjects - best runs:\nAccuracy = {:.4f}   Kappa = {:.4f}\n'.format(
        n_sub, np.average(acc_bestRun), np.average(kappa_bestRun)) 
    if(allRuns): 
        info = info + '\nAverage of {} subjects x {} runs (average of {} experiments):\nAccuracy = {:.4f}   Kappa = {:.4f}'.format(
            n_sub, acc_allRuns.shape[1], (n_sub * acc_allRuns.shape[1]),
            np.average(acc_allRuns), np.average(kappa_allRuns)) 
    print(info)
    log_write.write(info)
    
    draw_performance_barChart(n_sub, acc_bestRun, 'Accuracy')
    draw_performance_barChart(n_sub, kappa_bestRun, 'K-score')

    draw_confusion_matrix(cf_matrix.mean(0), 'All', results_path)

    log_write.close() 
    
    
#%%
def getModel(model_name):
    # Select the model
    if(model_name == 'ATCNet'):
       
        model = ATCNet( 
            # Dataset parameters
            n_classes = 4, 
            in_chans = 22, 
            in_samples = 1125, 
            # Sliding window (SW) parameter
            n_windows = 5, 
            # Attention (AT) block parameter
            attention = 'mha', # Options: None, 'mha','mhla', 'cbam', 'se'
            # Convolutional (CV) block parameters
            eegn_F1 = 16,
            eegn_D = 2, 
            eegn_kernelSize = 64,
            eegn_poolSize = 7,
            eegn_dropout = 0.3,
            # Temporal convolutional (TC) block parameters
            tcn_depth = 2, 
            tcn_kernelSize = 4,
            tcn_filters = 32,
            tcn_dropout = 0.3, 
            tcn_activation='elu'
            )     
    elif(model_name == 'TCNet_Fusion'):
        # Train using TCNet_Fusion: https://doi.org/10.1016/j.bspc.2021.102826
        model = TCNet_Fusion(n_classes = 4)      
    elif(model_name == 'EEGTCNet'):
        # Train using EEGTCNet: https://arxiv.org/abs/2006.00622
        model = EEGTCNet(n_classes = 4)          
    elif(model_name == 'EEGNet'):
        # Train using EEGNet: https://arxiv.org/abs/1611.08024
        model = EEGNet_classifier(n_classes = 4) 
    elif(model_name == 'EEGNeX'):
        # Train using EEGNeX: https://arxiv.org/abs/2207.12369
        model = EEGNeX_8_32(n_timesteps = 1000 , n_features = 22, n_outputs = 4)
    else:
        raise Exception("'{}' model is not supported model yet!".format(model_name))

    return model
    

def run():

    if not  os.path.exists(CFG.results_path):
      os.makedirs(CFG.results_path)

    #train()
    dataset_conf = { 'n_classes': 4, 'n_sub': 9, 'n_channels': 22, 'data_path': CFG.data_path,
                'isStandard': True, 'LOSO': False}
    test(getModel(CFG.model_name), dataset_conf, CFG.results_path)    
    
#%%
if __name__ == "__main__":
    run()

kernel_size: (None, 1125, 22, 8)
kernel_size: (None, 1125, 22, 8)
kernel_size: (None, 1125, 22, 8)
block1: (None, 1125, 22, 24)
block2: (None, 1125, 1, 48)
After pooling block2: (None, 160, 1, 48)
block3: (None, 160, 1, 32)
block3: (None, 22, 1, 32)
ATCNet block1 shape: (None, 22, 32)
result: (72, 1, 22, 1125)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 1011.14it/s]


result: (72, 1, 22, 1125)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 1075.73it/s]


result: (72, 1, 22, 1125)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 789.95it/s]


result: (72, 1, 22, 1125)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 1058.84it/s]


TRAINING_SHAPE: (446, 1, 22, 1125)
9/9 [==============================] - 1s 18ms/step


AttributeError: 'list' object has no attribute 'argmax'

In [ ]:
# np.load('/kaggle/working/embed_train_sub0.npy').shape